In [1]:
import pandas as pd
from Data.preprocess import preprocess, preprocess_sessions
from Models.LSTM_model import create_lstm_model, train_lstm_model, preprocess_for_lstm

In [2]:
# Load the data
drivers_csv = '../Data/drivers.csv'
weather_data = pd.read_csv('../Data/weather_data.csv')
intervals_data = pd.read_csv('../Data/interval_data.csv')
pit_data = pd.read_csv('../Data/pit_data.csv')
position_data = pd.read_csv('../Data/position_data.csv')
session_data = pd.read_csv('../Data/session_data.csv')
stint_data = pd.read_csv('../Data/stint_data.csv')
lap_data_files = [
    '../Data/lap_data_1.csv', '../Data/lap_data_2.csv', '../Data/lap_data_3.csv',
    '../Data/lap_data_4.csv', '../Data/lap_data_10.csv', '../Data/lap_data_11.csv',
    '../Data/lap_data_14.csv', '../Data/lap_data_16.csv', '../Data/lap_data_18.csv',
    '../Data/lap_data_20.csv', '../Data/lap_data_22.csv', '../Data/lap_data_23.csv',
    '../Data/lap_data_24.csv', '../Data/lap_data_27.csv', '../Data/lap_data_31.csv',
    '../Data/lap_data_44.csv', '../Data/lap_data_55.csv', '../Data/lap_data_63.csv',
    '../Data/lap_data_77.csv', '../Data/lap_data_81.csv'
]

In [3]:
# Preprocess the data
main_df = preprocess_sessions(drivers_csv, session_data)
main_df = preprocess(main_df, intervals_data, suffix='_intervals')
main_df = preprocess(main_df, pit_data, suffix='_pit')
main_df = preprocess(main_df, position_data, suffix='_position')
main_df = preprocess(main_df, stint_data, suffix='_stint')
main_df = preprocess(main_df, weather_data, suffix='_weather')

# Preprocess lap data
for i, lap_file in enumerate(lap_data_files):
    lap_data = pd.read_csv(lap_file)
    main_df = preprocess(main_df, lap_data, suffix=f'_lap{i+1}')

# Prepare data for LSTM
scaled_data, scaler = preprocess_for_lstm(main_df.values)
X_train, y_train = scaled_data[:, :-1], scaled_data[:, -1]

# Reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Create and train model
input_shape = (X_train.shape[1], X_train.shape[2])
model = create_lstm_model(input_shape)
model = train_lstm_model(model, X_train, y_train)

MemoryError: Unable to allocate 27.4 GiB for an array with shape (3679251756,) and data type int64